### VIDEO ENCODING LAB

If we want a video to be played in an adaptative vibrate streaming player we need differents qualities of the same file. We can achieve this wiht a huge variety of tools with or without user interface, privative or open sourced. We want the server to render the files by itself so the first part is understand and develop the necesary functions.


After some investigation in adaptative stream video players I've found that use ffmpeg for encoding the diffenents qualities and then uses shaka-packager to create the dash.manifest is a good idea.

So the next steps for our lab can be:

1. create a function to transcode the main video quality of each perspective to the webM(VP9) format recommened by google or MP4 codec (H264)
https://google.github.io/shaka-packager/html/tutorials/encoding.html

2. Package the qualities in the DASH specifications
https://google.github.io/shaka-packager/html/tutorials/dash.html



#### Video Rendering Tools

As reference in the following links we have two different open source projects that allow us to render files in a wide variety of codecs. They have available binary files in linux, windows and mac so you will need to download the apropiate one or built your own.

[GPAC is an Open Source multimedia framework](https://gpac.wp.imt.fr/)

[FFmpeg: convert/encode any audio/video](https://www.ffmpeg.org)

We are going to use python to call the ffmpeg tool which describes itself as an "Hyper fast Audio and Video encoder" in order to create the files.

### Steps

1. Check the tools
2. Analyze sample videos
3. Media Encoding
4. Analysis

***

### 1 - Check the tools


#### FFMPEG

we can get ffmpeg in the following [link](https://www.ffmpeg.org/)

#### TEST VIDEOS

Free available videos for test porpuses can be found [here](http://www.sample-videos.com/)

We are aiming to the MP4 and VP9 codecs.

In [6]:
## We are going to use ffmpeg library so we need to get the path in our local enviroment

import os
import sys
import pprint
import subprocess as sp

ffmpeg_path = os.path.join(os.getcwd(),'ffmpeg','bin', 'ffmpeg.exe')
ffprobe_path = os.path.join(os.getcwd(),'ffmpeg','bin', 'ffprobe.exe')
print(ffmpeg_path)
print(ffprobe_path)

D:\mp4dash\ffmpeg\bin\ffmpeg.exe
D:\mp4dash\ffmpeg\bin\ffprobe.exe


In [7]:
#In this cell we can check that we can call the ffmpeg tool from python using the subprocess module

#Run help from ffmped with subprocess
print(sp.check_output(ffmpeg_path + ' -h').decode())

# if worked after running this cell you should see the help message from ffmpeg

Hyper fast Audio and Video encoder
usage: ffmpeg [options] [[infile options] -i infile]... {[outfile options] outfile}...

Getting help:
    -h      -- print basic options
    -h long -- print more options
    -h full -- print all options (including all format and codec specific options, very long)
    -h type=name -- print all options for the named decoder/encoder/demuxer/muxer/filter
    See man ffmpeg for detailed description of the options.

Print help / information / capabilities:
-L                  show license
-h topic            show help
-? topic            show help
-help topic         show help
--help topic        show help
-version            show version
-buildconf          show build configuration
-formats            show available formats
-muxers             show available muxers
-demuxers           show available demuxers
-devices            show available devices
-codecs             show available codecs
-decoders           show available decoders
-encoders           

### 2 - Analyze sample videos

In [10]:
# get the test files

test_video_path_hq = os.path.join(os.getcwd(),'video_test','SampleVideo_1280x720_1mb.mp4')
test_video_path_mq = os.path.join(os.getcwd(),'video_test','SampleVideo_720x480_1mb.mp4')
test_video_path_sq = os.path.join(os.getcwd(),'video_test','SampleVideo_640x360_1mb.mp4')


In [33]:
# GETTING the METADATA of the videos

## the additional tool ffprobe is very usefull for this purpose.
import json
import pprint


def extract_metadata(video_file,ffprobe_path):
    # create the command for ffprobe
    metadata_command = '{} -print_format json -show_format -show_streams {}' \
                        .format(ffprobe_path,video_file)
    # call and decode the results
    info = sp.check_output(metadata_command).decode()
    # return the metadata in a json format
    return json.loads(info)


metadata = extract_metadata(test_video_path_mq,ffprobe_path)

# prety print the json entire object for visualization
# pprint.pprint(metadata)

# extract the wanted values
test_duration = metadata['streams'][0]['duration']

5.800000


## 3 - Media Encoding





In [36]:
import time

def convert_file(video_input_file, output_name, output_dir,ffmpeg_path,quality=0):
    #take the input an convert the file to the specify quality
    # video_input_file : the global path to the input file
    # output_name : the name for the output files
    # output_dir : the dir for the outputfiles
    
    # quality : default = 0 
    #         0 = 960x540
    #         1 = 650x360
    #         2 = 240x135
    
    quality_size = ['960x540','650x360','240x180']
    qualities = [540,360,180]
    vibrates = ['1000k', '500k', '100k']
    output = os.path.join(os.getcwd(), output_dir, output_name + str(quality_size[quality]) + '.mp4')
    
    command = ffmpeg_path + \
                ' -y -i ' + video_input_file + \
                " -c:a aac -ac 2 -ab 56k" + \
                " -c:v libx264 -x264opts keyint=24:min-keyint=24:no-scenecut" + \
                " -b:v " + vibrates[quality] + \
                " -vf scale=-1:" + str(qualities[quality]) + " " + \
                output
    # get the time
    start_time = time.time()
    print(start_time, ' converting ', command)
    # run the command
    command_output = sp.check_output(command).decode()
    print('command output: ',command_output)
    #calculate the elapsed_time
    elapsed_time = time.time() - start_time
    print('video conversion finalized ')
    print('elapsed_time: ', elapsed_time)
    
    return elapsed_time
    


In [37]:
elapsed_time_HQ = convert_file(test_video_path_hq, 'output_name', 'output_dir',ffmpeg_path,quality=0)

1513096575.5290854  converting  D:\mp4dash\ffmpeg\bin\ffmpeg.exe -y -i D:\mp4dash\video_test\SampleVideo_1280x720_1mb.mp4 -c:a aac -ac 2 -ab 56k -c:v libx264 -x264opts keyint=24:min-keyint=24:no-scenecut -b:v 1000k -vf scale=-1:540 D:\mp4dash\output_dir\output_name960x540.mp4
command output:  
video conversion finalized 
elapsed_time:  2.7079660892486572


In [38]:
elapsed_time_MQ = convert_file(test_video_path_hq, 'output_name', 'output_dir',ffmpeg_path,quality=1)

1513096578.2487454  converting  D:\mp4dash\ffmpeg\bin\ffmpeg.exe -y -i D:\mp4dash\video_test\SampleVideo_1280x720_1mb.mp4 -c:a aac -ac 2 -ab 56k -c:v libx264 -x264opts keyint=24:min-keyint=24:no-scenecut -b:v 500k -vf scale=-1:360 D:\mp4dash\output_dir\output_name650x360.mp4
command output:  
video conversion finalized 
elapsed_time:  1.6173858642578125


In [39]:
elapsed_time_SQ = convert_file(test_video_path_hq, 'output_name', 'output_dir',ffmpeg_path,quality=2)

1513096579.877145  converting  D:\mp4dash\ffmpeg\bin\ffmpeg.exe -y -i D:\mp4dash\video_test\SampleVideo_1280x720_1mb.mp4 -c:a aac -ac 2 -ab 56k -c:v libx264 -x264opts keyint=24:min-keyint=24:no-scenecut -b:v 100k -vf scale=-1:180 D:\mp4dash\output_dir\output_name240x180.mp4
command output:  
video conversion finalized 
elapsed_time:  0.9781577587127686


In [46]:
# function to adapt the WebM formats to python with subprocess
def convert_to_webM(video_input_file, output_dir, video_name ,ffmpeg_path):
    
    names = []
    paths = []
    commands = []
    
    #create the audio file commands
    audio_name = video_name + '_audio' + '.webm'
    audio_output = os.path.join(os.getcwd(), output_dir, audio_name)
    paths.append(audio_output)
    names.append(audio_name)
    
    # iterate over the qualities to create the video files commands
    audio_command = ffmpeg_path + \
                ' -y -i ' + video_input_file + \
                " -vn -acodec libvorbis -ab 128k -dash 1 " + \
                audio_output
    commands.append(audio_command)
    
#     scales = ['720','360','360','180','90']
#     vibrates = ['1500k','1000k', '750k', '500k', '250k']
    scales = ['540','360','180']
    vibrates = ['1000k', '500k', '100k']
    
    
    for i in range(0,len(scales)):
        video_name_ = video_name + '-' + scales[i] + '-' + vibrates[i] + '.webm'
        video_output = os.path.join(os.getcwd(), output_dir, video_name)
        video_command = ffmpeg_path + \
                    ' -y -i ' + video_input_file + \
                    " -c:v libvpx-vp9 -g 150 -tile-columns 4 -frame-parallel 1  -f webm -dash 1" + \
                    ' -an -vf scale=-1:' + scales[i] + ' -b:v '+ vibrates[i] + ' -dash 1 ' + \
                    video_output
                    
        paths.append(video_output)
        names.append(video_name_)
        commands.append(video_command)
        video_name_ = ''
        video_output = ''
        video_command = ''
        
    
    # LAUNCH the commmands and calculate the times
    times = []
    for j in range(0,len(commands)):
         # get the current time
        start_time = time.time()
        print(start_time, ' converting ', commands[j])
        # run the command
        command_output = sp.check_output(commands[j]).decode()
        print('command output: ',command_output)
        #calculate the elapsed_time
        elapsed_time = time.time() - start_time
        print('video conversion finalized ')
        print('elapsed_time: ', elapsed_time)
        times.append(elapsed_time)
    
    return times  

In [48]:
VP9_times = convert_to_webM(test_video_path_hq, 'output_dir', 'test_webm' ,ffmpeg_path)
print(VP9_times)

1513097093.783802  converting  D:\mp4dash\ffmpeg\bin\ffmpeg.exe -y -i D:\mp4dash\video_test\SampleVideo_1280x720_1mb.mp4 -vn -acodec libvorbis -ab 128k -dash 1 D:\mp4dash\output_dir\test_webm_audio.webm
command output:  
video conversion finalized 
elapsed_time:  0.7765512466430664
1513097094.5603533  converting  D:\mp4dash\ffmpeg\bin\ffmpeg.exe -y -i D:\mp4dash\video_test\SampleVideo_1280x720_1mb.mp4 -c:v libvpx-vp9 -g 150 -tile-columns 4 -frame-parallel 1  -f webm -dash 1 -an -vf scale=-1:540 -b:v 1000k -dash 1 D:\mp4dash\output_dir\test_webm
command output:  
video conversion finalized 
elapsed_time:  48.06491470336914
1513097142.625268  converting  D:\mp4dash\ffmpeg\bin\ffmpeg.exe -y -i D:\mp4dash\video_test\SampleVideo_1280x720_1mb.mp4 -c:v libvpx-vp9 -g 150 -tile-columns 4 -frame-parallel 1  -f webm -dash 1 -an -vf scale=-1:360 -b:v 500k -dash 1 D:\mp4dash\output_dir\test_webm
command output:  
video conversion finalized 
elapsed_time:  26.791942834854126
1513097169.4172108  conv

In [84]:
def hour_extrapolation(input_):
    seconds_1hour = 60*60
    extrap = input_ * seconds_1hour / test_duration
    return extrap

# SOME CALCULATIONS ON MP4 and VP9 codecs

print('MP4 encoding')
print('speed=', test_duration/elapsed_time_HQ, 'x')
print('speed=', test_duration/elapsed_time_MQ, 'x')
print('speed=', test_duration/elapsed_time_SQ, 'x')

time_for_1_perspective = elapsed_time_HQ + \
                         elapsed_time_MQ + \
                         elapsed_time_SQ


total_time_for_3_qualities = hour_extrapolation(elapsed_time_HQ) + \
                             hour_extrapolation(elapsed_time_MQ) + \
                             hour_extrapolation(elapsed_time_SQ) 
        
print('total time for 1 hour video = ',total_time_for_3_qualities/3600, 'h')
print(time_for_1_perspective, 's')

print('VP9 encoding')
print('HQ_speed=', test_duration/VP9_times[3], 'x')
print('MQ_speed=', test_duration/VP9_times[2], 'x')
print('SQ_speed=', test_duration/VP9_times[1], 'x')

time_for_1_perspective = VP9_times[3] + \
                         VP9_times[2] + \
                         VP9_times[1]


total_time_for_3_qualities = hour_extrapolation(VP9_times[3]) + \
                             hour_extrapolation(VP9_times[2]) + \
                             hour_extrapolation(VP9_times[1]) 
        
print('total time for 1 hour video = ',total_time_for_3_qualities/3600, 'h')
print(time_for_1_perspective, 's')

MP4 encoding
speed= 2.14182888885778 x
speed= 3.586033566987745 x
speed= 5.929513872724024 x
total time for 1 hour video =  0.9143982262446964 h
5.303509712219238 s
VP9 encoding
HQ_speed= 0.8181388934711059 x
MQ_speed= 0.21648299400126647 x
SQ_speed= 0.12067014028412382 x
total time for 1 hour video =  14.12864109565472 h
81.94611835479736 s


### 4 - Analysis

The following table present an overview of the result

| resolution    | MP4 vibrate       | MP4 conversion rate       |  VP9 vibrate       | VP9 conversion rate       | 
| ------------- |:-------------:| :-------------:| :-------------:| :-------------:|
| 360| 100 |  5.05x |100 | 0.810x |
 | 480| 500 |  3.58x| 500 | | 0.21 |
 | 720| 1000 |  2.14x |1000  | 0.12 |
 
 


### NOTES

- For the same quality VP9 transcoding seems to be much more slower than H264, we should git it a try in a production computer to see if it can manage the VP9 codec better than my computer

- WebM  is an open, royalty-free media file format 

##### REFERENCES:

- http://wiki.webmproject.org/adaptive-streaming/instructions-to-playback-adaptive-webm-using-dash
- https://trac.ffmpeg.org/wiki/Encode/VP9
- https://developers.google.com/media/vp9/
- https://github.com/google/shaka-packager
- https://developers.google.com/media/pdf/Youtube-HTML5-VP9-Case-Study.pdf

In [19]:
def google_mp4_format_commands(video_input_file_path, output_dir_path, output_name, ffmpeg_path):
    commands = {}

    commands['360'] = ffmpeg_path + \
                    ' -y -i ' +  video_input_file_path + ' -c:a copy ' + \
                    ' -vf "scale=-2:360" ' + \
                    ' -c:v libx264 -profile:v baseline -level:v 3.0 ' + \
                    ' -x264opts scenecut=0:open_gop=0:min-keyint=72:keyint=72 '+ \
                    ' -minrate 600k -maxrate 600k -bufsize 600k -b:v 600k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + '_baseline_360p_600.mp4'
                    
    commands['480'] = ffmpeg_path + \
                    ' -i ' +  video_input_file_path + ' -c:a copy ' + \
                    ' -vf "scale=-2:480" ' + \
                    ' -c:v libx264 -profile:v baseline -level:v 3.0 ' + \
                    ' -x264opts scenecut=0:open_gop=0:min-keyint=72:keyint=72 '+ \
                    ' -minrate 1000k -maxrate 1000k -bufsize 1000k -b:v 1000k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + '_baseline_480p_1000.mp4'
                    
    commands['720'] = ffmpeg_path + \
                    ' -i ' +  video_input_file_path + ' -c:a copy ' + \
                    ' -vf "scale=-2:720" ' + \
                    ' -c:v libx264 -profile:v baseline -level:v 3.0 ' + \
                    ' -x264opts scenecut=0:open_gop=0:min-keyint=72:keyint=72 '+ \
                    ' -minrate 1500k -maxrate 1500k -bufsize 1500k -b:v 1500k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + '_baseline_720p_1500.mp4'
                    
    commands['1080'] = ffmpeg_path + \
                    ' -i ' +  video_input_file_path + ' -c:a copy ' + \
                    ' -vf "scale=-2:1080" ' + \
                    ' -c:v libx264 -profile:v baseline -level:v 3.0 ' + \
                    ' -x264opts scenecut=0:open_gop=0:min-keyint=72:keyint=72 '+ \
                    ' -minrate 3000k -maxrate 3000k -bufsize 3000k -b:v 3000k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + '_baseline_1080p_3000.mp4'
    
    
    return commands

def google_v9_format_commands(video_input_file_path, output_dir_path, output_name, ffmpeg_path):
    commands = {}

    commands['360'] = ffmpeg_path + \
                    ' -i ' +  video_input_file_path +  \
                    ' -strict -2 -c:a libvorbis ' + \
                    ' -vf "scale=-2:360" ' + \
                    ' -c:v libvpx-vp9 -profile:v 0 ' + \
                    ' -keyint_min 72 -g 72 ' + \
                    ' -tile-columns 4 -frame-parallel 1 -speed 1 ' + \
                    ' -auto-alt-ref 1 -lag-in-frames 25 ' + \
                    ' -b:v 300k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + 'vp9_360p_300.webm'
                    

    commands['480'] = ffmpeg_path + \
                    ' -i ' +  video_input_file_path +  \
                    ' -strict -2 -c:a libvorbis ' + \
                    ' -vf "scale=-2:480" ' + \
                    ' -c:v libvpx-vp9 -profile:v 0 ' + \
                    ' -keyint_min 72 -g 72 ' + \
                    ' -tile-columns 4 -frame-parallel 1 -speed 1 ' + \
                    ' -auto-alt-ref 1 -lag-in-frames 25 ' + \
                    ' -b:v 500k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + 'vp9_480p_500.webm'
                    
    commands['720'] = ffmpeg_path + \
                    ' -i ' +  video_input_file_path +  \
                    ' -strict -2 -c:a libvorbis ' + \
                    ' -vf "scale=-2:720" ' + \
                    ' -c:v libvpx-vp9 -profile:v 0 ' + \
                    ' -keyint_min 72 -g 72 ' + \
                    ' -tile-columns 4 -frame-parallel 1 -speed 1 ' + \
                    ' -auto-alt-ref 1 -lag-in-frames 25 ' + \
                    ' -b:v 1500k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + 'vp9_720p_1500.webm'
                    
    commands['1080'] = ffmpeg_path + \
                    ' -i ' +  video_input_file_path +  \
                    ' -strict -2 -c:a libvorbis ' + \
                    ' -vf "scale=-2:1080" ' + \
                    ' -c:v libvpx-vp9 -profile:v 0 ' + \
                    ' -keyint_min 72 -g 72 ' + \
                    ' -tile-columns 4 -frame-parallel 1 -speed 1 ' + \
                    ' -auto-alt-ref 1 -lag-in-frames 25 ' + \
                    ' -b:v 3000k ' + \
                    ' -y ' +  os.path.join(output_dir_path,output_name) + 'vp9_1080p_3000.webm'
    
    
    return commands

In [30]:
output_dir_path = os.path.join(os.getcwd(),'output_dir')
output_name = 'google_'

commands = google_mp4_format_commands(test_video_path_hq,output_dir_path,output_name,ffmpeg_path)
print(commands['1080'])

D:\mp4dash\ffmpeg\bin\ffmpeg.exe -i D:\mp4dash\video_test\TG1a-hd.mp4 -c:a copy  -vf "scale=-2:1080"  -c:v libx264 -profile:v baseline -level:v 3.0  -x264opts scenecut=0:open_gop=0:min-keyint=72:keyint=72  -minrate 3000k -maxrate 3000k -bufsize 3000k -b:v 3000k  -y D:\mp4dash\output_dir\google__baseline_1080p_3000.mp4


In [31]:
## RUNNING THE FOUR QUALITIES OVER THE MP4 GOOGLE RECOMENDATIONS

for key, value in commands.items():
    # get the time
    start_time = time.time()
    # run the command
    command_output = sp.check_output(value).decode()
    #calculate the elapsed_time
    elapsed_time = time.time() - start_time
    print('video conversion finalized  for encoding', key)
    print('elapsed_time: ', elapsed_time)

# print(sp.check_output(commands['360']).decode())

video conversion finalized  for encoding 360
elapsed_time:  475.7102267742157
video conversion finalized  for encoding 480
elapsed_time:  769.1079740524292
video conversion finalized  for encoding 720
elapsed_time:  1186.839587211609
video conversion finalized  for encoding 1080
elapsed_time:  2171.9001331329346
